In [1]:
from lxml import html
from bs4 import BeautifulSoup
import requests
import urllib.request
import bs4
import csv
import pandas as pd

In [3]:
# create list of the links in movies
tree = html.parse(r"movies2.html")
movies2 = html.tostring(tree)
soup = BeautifulSoup(movies2)
links = []
for link in soup.findAll('a'):
    links.append(link.get('href'))
links

['https://en.wikipedia.org/wiki/10_to_Midnight',
 'https://en.wikipedia.org/wiki/All_the_Right_Moves_(film)',
 'https://en.wikipedia.org/wiki/Americana_(film)',
 'https://en.wikipedia.org/wiki/Amityville_3-D',
 'https://en.wikipedia.org/wiki/Anna_to_the_Infinite_Power',
 'https://en.wikipedia.org/wiki/Baby_It%27s_You_(film)',
 'https://en.wikipedia.org/wiki/Bad_Boys_(1983_film)',
 'https://en.wikipedia.org/wiki/Better_Late_Than_Never_(film)',
 'https://en.wikipedia.org/wiki/The_Big_Chill_(film)',
 'https://en.wikipedia.org/wiki/The_Black_Stallion_Returns',
 'https://en.wikipedia.org/wiki/Blue_Thunder',
 'https://en.wikipedia.org/wiki/Born_in_Flames',
 'https://en.wikipedia.org/wiki/Brainstorm_(1983_film)',
 'https://en.wikipedia.org/wiki/Breathless_(1983_film)',
 'https://en.wikipedia.org/wiki/Can_She_Bake_a_Cherry_Pie%3F',
 'https://en.wikipedia.org/wiki/Chicken_Ranch_(film)',
 'https://en.wikipedia.org/wiki/Christine_(1983_film)',
 'https://en.wikipedia.org/wiki/A_Christmas_Story',
 

In [4]:
for i in range(10000, 10000 + len(links)+1):
    try:
        html = urllib.request.urlopen(links[i-10000]).read() 
        with open('../data_input/article_{}.html'.format(i), 'wb+') as f:
                    f.write(html)
    except Exception as e:
        print(i, links[i-10000], e)
        continue

15520 https://en.wikipedia.org/wiki/Boogie_Town HTTP Error 404: Not Found
15576 https://en.wikipedia.org/wiki/Horrorween HTTP Error 404: Not Found
17725 https://en.wikipedia.org/wiki/Offside_(2009_film) HTTP Error 404: Not Found
18100 https://en.wikipedia.org/wiki/Say_It_with_Music_(film) HTTP Error 404: Not Found


IndexError: list index out of range

In [35]:
# save all .tsv
# made some test
for article in range(20):  # range of files
    try:
        html = BeautifulSoup(open("../data_input/article_{}.html".format(article)), 'html.parser')
    except Exception as e:   # if article doesn't exists
        print(article, e)
        continue

    title = html.select("h1")[0].text
        
    # initialize tmp as intro
    tmp = 'intro'
    sections = {'intro' : '', 'plot' : ''}
    
    # take all paragraphs section by section and save only intro and plot
    for section in html.select('div.mw-parser-output > *'):    # take only notes in the first level
        if(section.name == 'p' and tmp == 'intro'):
            sections['intro'] += section.text.strip()
    
        # chage tmp on section names
        if(section.name in ['h2','h3']):
            tmp = section.span['id']
    
        # take only sections we are interrested in
        if(section.name == 'p' and tmp in ['Plot','Plot_summary','Premiese']):  # check different names for plot sections
            sections['plot'] += section.text.strip()
    
    # we doesn't take in consideration pages without Plot
    if(sections['plot'] == ''):
        print(article, 'No Plot')
        continue

    # dictionary for infobox
    d = {'film_name':title, 'Directed by': 'NA', 'Produced by': 'NA', 'Written by': 'NA', 'Starring':'NA', 'Music by': 'NA',
         'Release date': 'NA', 'Running time': 'NA', 'Country': 'NA', 'Language': 'NA', 'Budget': 'NA'}

    # take elem from infobox
    info_box = html.findAll(['th', 'td'])
    for elem in info_box:
        info = elem.text.strip('\n')   # take text from the table
        if info in d:
            d[info] = info_box[info_box.index(elem)+1].text.strip('\n')
    
    # select elem in oroder as a list to save in .tsv
    ld = list(d.values())
    columns = ['title', 'intro', 'plot', 'film_name', 'Directed by', 'Produced by', 'Written by', 'Starring', 'Music by',
         'Release date', 'Running time', 'Country', 'Language', 'Budget']
    data = [title, sections['intro'], sections['plot']] + ld[0:]
    # create and save a tsv
    with open('../data_output/article_{}.tsv'.format(article), 'w', newline='',encoding='utf-8') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t')
        tsv_output.writerow(columns)
        tsv_output.writerow(data)

7 No Plot


In [36]:
b = pd.read_csv('../data_output/article_0.tsv', sep = '\t')
for i in range(1, 20):
    try:
        b = b.append(pd.read_csv('../data_output/article_{}.tsv'.format(i), sep = '\t'), ignore_index = True)
    except Exception as e:
        print(i, e)
b

7 [Errno 2] File b'../data_output/article_7.tsv' does not exist: b'../data_output/article_7.tsv'


,title,intro,plot,film_name,Directed by,Produced by,Written by,Starring,Music by,Release date,Running time,Country,Language,Budget
0,10 to Midnight,10 to Midnight is a 1983 American crime-horror...,Warren Stacy (Gene Davis) is a young office eq...,10 to Midnight,J. Lee Thompson,Pancho KohnerLance Hool,William RobertsJ. Lee Thompson,Charles Bronson\nLisa Eilbacher\nAndrew Steven...,Robert O. Ragland,"March 11, 1983 (1983-03-11) (U.S.)",101 min,United States,English,"$4,520,000 (US)"
1,All the Right Moves (film),All the Right Moves is a 1983 American sports ...,"Stefen ""Stef"" Djordjevic (Cruise) is a Serbian...",All the Right Moves (film),Michael Chapman,Stephen Deutsch,Michael Kane,Tom Cruise\nCraig T. Nelson\nLea Thompson,David Richard Campbell,"October 21, 1983 (1983-10-21)",91 minutes,United States,English,$5.6 million[1]
2,Americana (film),Americana is a 1983 American drama film starri...,"A down on his luck, former Green Beret captain...",Americana (film),David Carradine,David CarradineSkip Sherwood,Henry Morton Robinson,David CarradineBarbara HersheyMichael Greene,Craig Huntley,"October 21, 1983 (1983-10-21)",91 min.,United States,English,NaN
3,Amityville 3-D,Amityville 3-D (also known as Amityville III: ...,After he exposes a pair of con artists with hi...,Amityville 3-D,Richard Fleischer,Stephen F. Kesten,David Ambrose,Tony Roberts\nTess Harper\nRobert Joy\nCandy C...,Howard Blake,"November 18, 1983 (1983-11-18)",105 min (US)93 min. (UK),United StatesMexico,English,$6 million
4,Anna to the Infinite Power,Anna to the Infinite Power is a 1982 science-f...,"Twelve-year-old Anna Hart of Flemington, New J...",Anna to the Infinite Power,Robert Wiemer,Bruce Graham,Robert WiemerBased on Anna to the Infinite Pow...,Martha ByrneDina MerrillMark PattonDonna Mitch...,Paul Baillargeon,1982[1],105 min.[1],United States,English,NaN
5,Baby It's You (film),Baby It's You is a 1983 American romantic come...,"The film, set in 1966 New Jersey, is about a r...",Baby It's You (film),John Sayles,Griffin DunneAmy Robinson,NaN,Rosanna Arquette\nVincent Spano,Todd Kasow,"March 4, 1983 (1983-03-04) (United States)",105 minutes,United States,English,"$3,000,000[1]"
6,Bad Boys (1983 film),Bad Boys is a 1983 American coming-of-age neo-...,Mick O'Brien is a 16-year-old Irish-American h...,Bad Boys (1983 film),Rick Rosenthal,Robert H. Solo,Richard Di Lello,Sean Penn\nReni Santoni\nAlly Sheedy\nJim Mood...,Bill Conti,"March 25, 1983 (1983-03-25)",123 minutes\n104 minutes (Edited cut),United States,English,$5 million[citation needed]
7,The Big Chill (film),The Big Chill is a 1983 American comedy-drama ...,Harold Cooper is bathing his young son when hi...,The Big Chill (film),Lawrence Kasdan,Michael Shamberg,Lawrence KasdanBarbara Benedek,Tom Berenger\nGlenn Close\nJeff Goldblum\nWill...,Bill Conti,"September 28, 1983 (1983-09-28)",105 minutes,United States,English,$8 million[1]
8,The Black Stallion Returns,The Black Stallion Returns is a 1983 film adap...,The story begins with several odd occurrences ...,The Black Stallion Returns,Robert Dalva,Fred RoosFrancis Ford CoppolaTom Sternberg,NaN,Kelly Reno\nTeri Garr,Georges Delerue,"March 25, 1983 (1983-03-25)",103 minutes,United States,English,NaN
9,Blue Thunder,Blue Thunder is a 1983 action thriller film fr...,Frank Murphy (Roy Scheider) is a Los Angeles P...,Blue Thunder,John Badham,Gordon CarrollPhil FeldmanAndrew Fogelson,Dan O'BannonDon Jakoby,Roy Scheider\nWarren Oates\nCandy Clark\nDanie...,Arthur B. Rubinstein,"May 13, 1983 (1983-05-13)",109 minutes,United States,English,$11 million
